In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import html
import pandas as pd
import datetime
import time
import random
import warnings
import ast

from tqdm import tqdm

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [2]:
date_to_run = '20230604'
df = pd.read_csv(f'xe_{date_to_run}.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5833 entries, 0 to 5832
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5833 non-null   int64  
 1   internal_id              5833 non-null   int64  
 2   item_type                5833 non-null   object 
 3   transaction_type         5833 non-null   object 
 4   address                  5833 non-null   object 
 5   geo_lat                  5833 non-null   float64
 6   geo_lng                  5833 non-null   float64
 7   ad_group_id              743 non-null    float64
 8   levels                   5833 non-null   object 
 9   bedrooms                 5546 non-null   float64
 10  bathrooms                5267 non-null   float64
 11  construction_year        4745 non-null   float64
 12  display_golden_border    5833 non-null   bool   
 13  owner_logo               4481 non-null   object 
 14  price_per_unit_area     

In [4]:
df.describe(include = ['object', 'bool'])

,item_type,transaction_type,address,levels,display_golden_border,owner_logo,company_title,is_commercial,sponsored,title,title_abbreviation,price,price_with_abbreviation,price_per_square_meter,size_with_square_meter,date,extra_seo_info,image_alt,image_gallery,url,unique_group_url,extraction_date
count,5833,5833,5833,5833,5833,4481,4482,5833,5833,5833,5833,5833,5833,5830,5833,2645,5833,5833,5833,5833,743,5833
unique,1,1,234,62,2,576,574,2,1,422,422,256,253,55,285,29,1,4859,5145,5830,743,1
top,re_residence,LET.NORMAL,Αθήνα (Κολωνάκι),['1ος'],False,https://a2.xe.gr/LOGO/1622120851959.jpg,Golden Home,True,False,Διαμέρισμα 50 τ.μ.,Διαμέρισμα 50 τ.μ.,450 €,450 €,8 €/τ.μ.,50 τ.μ.,πριν από 2 εβδομάδες,Ενοικίαση κατοικίας,Ενοικίαση κατοικίας Αθήνα (Κολωνάκι) Διαμέρισμ...,[],https://www.xe.gr/property/d/enoikiaseis-katoi...,https://www.xe.gr/property/u/460563,2023-06-04
freq,5833,5833,240,1251,5350,315,315,4557,5833,285,285,322,322,818,287,581,5833,7,687,2,1,5833


In [5]:
df.describe()

,id,internal_id,geo_lat,geo_lng,ad_group_id,bedrooms,bathrooms,construction_year,price_per_unit_area,account_id
count,5.833000e+03,5.833000e+03,5833.000000,5833.000000,743.000000,5546.000000,5267.000000,4745.000000,5833.000000,5.741000e+03
mean,7.955918e+08,4.642796e+07,37.986653,23.739844,428011.590848,1.863866,1.193469,1976.241939,11.657980,1.205480e+06
std,6.928650e+07,2.074199e+06,0.019568,0.020638,52625.936069,1.228180,0.734358,16.009774,50.177156,7.219027e+05
min,5.765660e+05,2.009286e+07,37.948830,23.687081,55188.000000,1.000000,1.000000,1900.000000,0.000000,2.003400e+04
25%,8.045262e+08,4.657052e+07,37.971767,23.727107,427874.500000,1.000000,1.000000,1970.000000,8.000000,5.301200e+05
50%,8.124922e+08,4.699751e+07,37.986548,23.739453,446053.000000,2.000000,1.000000,1975.000000,9.000000,1.286305e+06
75%,8.143664e+08,4.711165e+07,38.000705,23.753701,455157.500000,2.000000,1.000000,1980.000000,12.000000,1.879279e+06
max,8.154122e+08,4.716078e+07,38.032695,23.789568,460571.000000,21.000000,16.000000,2023.000000,3205.000000,2.268637e+06


In [6]:
useless_columns = ['item_type', 'transaction_type', 'owner_logo', 'title', 
                   'price_with_abbreviation', 'extra_seo_info', 'image_alt', 
                   'image_gallery', 'unique_group_url', 'price_per_square_meter', 'display_golden_border',
                   'is_commercial', 'sponsored']

In [7]:
useful_columns = ['id', 'internal_id', 'title_abbreviation','address','price','price_per_unit_area',
                  'size_with_square_meter','construction_year','levels','bedrooms', 'bathrooms',  
                   'date', 'extraction_date', 'geo_lat', 'geo_lng','company_title', 'account_id','ad_group_id', 'url']

In [8]:
len(useless_columns)+len(useful_columns)

32

In [9]:
df_useful = df.loc[:, useful_columns]

In [10]:
df_useful[['dimos', 'perioxi']] = df_useful['address'].str.extract(r'([\w\s]+)(?:\(([\w\s]+)\))?')
df_useful['dimos'] = df_useful['dimos'].str.strip()

In [11]:
dimoi_df = pd.read_csv('dimoi.csv')
dimoi_dict = dimoi_df.set_index('Raw')['Fixed'].to_dict()
missing_dimoi = set(df_useful['dimos'].unique()) - set(dimoi_dict.keys())
if missing_dimoi: 
    warnings.warn(f'The following dimoi are new: {missing_dimoi}')
df_useful['dimos'] = df_useful['dimos'].map(dimoi_dict)

In [12]:
df_useful['type'] = df_useful['title_abbreviation'].str.extract("(\w+)")
df_useful['price'] = df_useful['price'].str.replace('[.,]', '', regex=True).str.extract(r'(\d+)').astype('float')
df_useful['area'] = df['size_with_square_meter'].str.replace('[.,]', '', regex=True).str.extract(r'(\d+)').astype('float')
df_useful['price_per_area'] = df_useful['price'] / df_useful['area']
df_useful['date'] = df_useful['date'].fillna('πριν από 2 μήνες')

In [13]:
df_useful['levels'] = df_useful['levels'].str.replace('Υπόγειο', '-1').str.replace('Ισόγειο', '0').str.replace('Υπερυψωμένο', '0.5').str.replace('Ημιυπόγειο', '-0.5').str.replace('Ημιώροφος', '0.5').str.replace('ος', '')
df_useful['level_list'] = df_useful['levels'].apply(ast.literal_eval).apply(lambda lst: [float(element) for element in lst])

def get_smallest_level(level_list):
    if not level_list: return None
    min_val = min(level_list)
    if min_val < 0:
        non_negatives = [num for num in level_list if num >= 0]
        if non_negatives:
            min_val = min(non_negatives)
    return min_val
df_useful['min_level'] = df_useful['level_list'].apply(get_smallest_level)
df_useful['level_length'] = df_useful['level_list'].apply(len)

In [14]:
df_useful.isnull().sum()

id                           0
internal_id                  0
title_abbreviation           0
address                      0
price                        0
price_per_unit_area          0
size_with_square_meter       0
construction_year         1088
levels                       0
bedrooms                   287
bathrooms                  566
date                         0
extraction_date              0
geo_lat                      0
geo_lng                      0
company_title             1351
account_id                  92
ad_group_id               5090
url                          0
dimos                        0
perioxi                    306
type                         0
area                         0
price_per_area               0
level_list                   0
min_level                    2
level_length                 0
dtype: int64

In [15]:
df_useful[['temp_no', 'temp_date_unit']] = df_useful['date'].str.extract(r'(\d+)\s+(\w{2})')

In [16]:
df_useful['days_passed'] = 0
df_useful['days_passed'][df_useful['temp_date_unit'] == "ημ"] = 1 * df_useful['temp_no'].astype(float)
df_useful['days_passed'][df_useful['temp_date_unit'] == "εβ"] = 7 * df_useful['temp_no'].astype(float)
df_useful['days_passed'][df_useful['temp_date_unit'] == "μή"] = 30 * df_useful['temp_no'].astype(float)

In [17]:
df_clean = df_useful[(4 <= df_useful['price_per_area']) & (df_useful['price_per_area'] <= 25) & 
                     (df_useful['price'] >= 100) & 
                     (df_useful['type'] != "Κτίριο") & 
                     (df_useful['dimos'] != "WRONG") & 
                     (df_useful['level_length'] <= 3) &
                     ~((df_useful['type'] != "Διαμέρισμα") & (df_useful['level_length'] != 1))]

In [18]:
df_clean.to_csv(f'xe_clean_{date_to_run}.csv', encoding='utf-8-sig', index=False)

In [19]:
print(f"Original: {len(df_useful)} rows.\nFiltered: {len(df_clean)} rows.\nRemoved: {len(df_useful) - len(df_clean)} rows.")

Original: 5833 rows.
Filtered: 5457 rows.
Removed: 376 rows.
